In [1]:
import os
import sys

os.chdir('./')
print("Current working directory: {0}".format(os.getcwd()))
sys.path.append(os.getcwd() + '/src')

Current working directory: /Users/paulinamajecka/Documents/GitHub/coding_exercises/python/part2/solutions/pmajecka


In [2]:
import pandas as pd
import plotly.graph_objects as go
from pathlib import Path
from plotly.subplots import make_subplots

#### 1. Read data from given csv files, merge it by the “time” column, create a datetime index with the column “time”.

In [3]:
data_path = '../../data/'

In [4]:
a = pd.read_csv(data_path + 'data_a.csv')
a.head(5)

,time,channel_a
0,2023-02-15T13:50:00,NaN
1,2023-02-15T13:55:00,NaN
2,2023-02-15T14:00:00,NaN
3,2023-02-15T14:05:00,230.210007
4,2023-02-15T14:10:00,224.690002


In [5]:
b = pd.read_csv(data_path + 'data_b.csv', parse_dates=True)
b.head(5)

,time,channel_b
0,2023-02-09T12:35:00,148.800003
1,2023-02-13T10:30:00,138.429993
2,2023-02-13T10:35:00,137.820007
3,2023-02-13T10:40:00,136.589996
4,2023-02-14T07:55:00,NaN


In [6]:
c = pd.read_csv(data_path + 'data_c.csv', parse_dates=True)
c.head(5)

,time,channel_c
0,2023-03-01T14:50:00,NaN
1,2023-03-01T14:55:00,NaN
2,2023-03-01T15:00:00,NaN
3,2023-03-01T15:05:00,303.329987
4,2023-03-01T15:15:00,195.830002


In [7]:
data = a.merge(b, how='outer', on='time')
data = data.merge(c, how='outer', on='time')

data.head()

,time,channel_a,channel_b,channel_c
0,2023-02-15T13:50:00,NaN,353.769989,NaN
1,2023-02-15T13:55:00,NaN,353.779999,NaN
2,2023-02-15T14:00:00,NaN,349.609985,NaN
3,2023-02-15T14:05:00,230.210007,355.950012,NaN
4,2023-02-15T14:10:00,224.690002,354.839996,NaN


In [8]:
data['time'] = pd.to_datetime(data['time'])
data.set_index('time', inplace=True)
data = data.resample('5min').asfreq()

data.head(5)

,channel_a,channel_b,channel_c
time,,,
2023-02-09 12:35:00,NaN,148.800003,NaN
2023-02-09 12:40:00,NaN,NaN,NaN
2023-02-09 12:45:00,NaN,NaN,NaN
2023-02-09 12:50:00,NaN,NaN,NaN
2023-02-09 12:55:00,NaN,NaN,NaN


#### 2. Check basic information about the data (e.g. type, size, missing values)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 81697 entries, 2023-02-09 12:35:00 to 2023-11-20 04:35:00
Freq: 5T
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   channel_a  78505 non-null  float64
 1   channel_b  80265 non-null  float64
 2   channel_c  75560 non-null  float64
dtypes: float64(3)
memory usage: 2.5 MB


In [10]:
data.describe()

,channel_a,channel_b,channel_c
count,78505.000000,80265.000000,75560.000000
mean,179.408043,358.319335,191.336256
std,30.439674,63.111887,29.145443
min,100.000000,0.000000,97.589996
25%,160.020004,316.420013,170.559998
50%,182.199997,364.950012,196.360001
75%,198.919998,401.899994,213.679993
max,322.959991,575.570007,303.329987


#### 3. Suggest a few ways to get rid of missing values (e.g., delete, replace with another value). 

1. Simple interpolation

In [11]:
interpolated_data = data.interpolate(method='linear', limit_direction='forward', axis=0)
interpolated_data.head()

,channel_a,channel_b,channel_c
time,,,
2023-02-09 12:35:00,NaN,148.800003,NaN
2023-02-09 12:40:00,NaN,148.790802,NaN
2023-02-09 12:45:00,NaN,148.781600,NaN
2023-02-09 12:50:00,NaN,148.772399,NaN
2023-02-09 12:55:00,NaN,148.763197,NaN


2. Replacing nulls with value e.g. 0

In [12]:
filled_data = data.fillna(0)
filled_data.head()

,channel_a,channel_b,channel_c
time,,,
2023-02-09 12:35:00,0.0,148.800003,0.0
2023-02-09 12:40:00,0.0,0.000000,0.0
2023-02-09 12:45:00,0.0,0.000000,0.0
2023-02-09 12:50:00,0.0,0.000000,0.0
2023-02-09 12:55:00,0.0,0.000000,0.0


3. Removing rows from begining until all chanels have values and interpolation nulls in the middle.

In [13]:
any_null_in_row = data.isna().any(axis=1)
first_not_null_row = any_null_in_row[any_null_in_row==False].index.min()
removed_data = data[data.index>=first_not_null_row]
removed_data = removed_data.interpolate(method='linear', limit_direction='forward', axis=0)
removed_data.head()

,channel_a,channel_b,channel_c
time,,,
2023-03-01 15:05:00,156.779999,328.070007,303.329987
2023-03-01 15:10:00,157.539993,322.290009,249.579994
2023-03-01 15:15:00,158.429993,329.019989,195.830002
2023-03-01 15:20:00,158.710007,326.600006,197.289993
2023-03-01 15:25:00,159.100006,326.529999,199.309998


#### 4. Make plots for all channels in data.

In [14]:
path_to_save_plot = './plots/'
Path(path_to_save_plot).mkdir(parents=True, exist_ok=True)

Channel 'a' plot

In [15]:
fig = go.Figure()
fig.add_traces(go.Scatter(x=data.index, y=data['channel_a'], mode='lines', name='channel_a'))
fig.update_layout(
    title={'text':"Channel 'a' data", 'y':0.95, 'x':0.5, 'xanchor': 'center', 'font':dict(size=25)}, 
    xaxis_title={'text':'Time'}
    )
fig.write_html(path_to_save_plot + 'channel_a.html')

Channel 'b' plot

In [16]:
fig = go.Figure()
fig.add_traces(go.Scatter(x=data.index, y=data['channel_b'], mode='lines', name='channel_b'))
fig.update_layout(
    title={'text':"Channel 'b' data", 'y':0.95, 'x':0.5, 'xanchor': 'center', 'font':dict(size=25)}, 
    xaxis_title={'text':'Time'}
    )
fig.write_html(path_to_save_plot + 'channel_b.html')

Channel 'c' plot

In [17]:
fig = go.Figure()
fig.add_traces(go.Scatter(x=data.index, y=data['channel_c'], mode='lines', name='channel_c'))
fig.update_layout(
    title={'text':"Channel 'c' data", 'y':0.95, 'x':0.5, 'xanchor': 'center', 'font':dict(size=25)}, 
    xaxis_title={'text':'Time'}
    )
fig.write_html(path_to_save_plot + 'channel_c.html')

All channels plot 

In [18]:
channels_list = ['channel_a', 'channel_b', 'channel_c']
fig = go.Figure()
for channel_name in channels_list:
    fig.add_traces(go.Scatter(x=data.index, y=data[channel_name], mode='lines', name=channel_name))
fig.update_layout(
    title={'text':"All channels in data", 'y':0.95, 'x':0.5, 'xanchor': 'center', 'font':dict(size=25)}, 
    xaxis_title={'text':'Time'}
    )
fig.write_html(path_to_save_plot + 'data.html')

All channels subplot

In [19]:
channels_list = ['channel_a', 'channel_b', 'channel_c']
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05, subplot_titles=channels_list)
for i, channel_name in enumerate(channels_list):
    fig.append_trace(go.Scatter(x=data.index, 
                                y=data[channel_name], 
                                mode='lines', 
                                name=channel_name), 
                                row=1+i, col=1)
fig.update_layout(
    title={'text':"All channels in data", 'y':0.98, 'x':0.45, 'xanchor': 'center', 'font':dict(size=25)}
    )
fig.update_xaxes(title_text='time', row=1+i, col=1)
fig.write_html(path_to_save_plot + 'data_subplot.html')
    


#### 5. Print mean values for grouped data by the hour, month, year. Try other statistical functions.

Group by year with mean, max and std

In [20]:
temp_year = data.copy()
temp_year['year'] = data.index.year

groupby_year_data = pd.DataFrame()
groupby_year_data[data.columns+'_mean'] = temp_year.groupby(['year']).mean()
groupby_year_data[data.columns+'_max'] = temp_year.groupby(['year']).mean()
groupby_year_data[data.columns+'_std'] = temp_year.groupby(['year']).mean()
groupby_year_data

,channel_a_mean,channel_b_mean,channel_c_mean,channel_a_max,channel_b_max,channel_c_max,channel_a_std,channel_b_std,channel_c_std
year,,,,,,,,,
2023,179.408043,358.319335,191.336256,179.408043,358.319335,191.336256,179.408043,358.319335,191.336256


Group by month with mean, max and std

In [21]:
temp_month = data.copy()
temp_month['month'] = data.index.month

groupby_month_data = pd.DataFrame()
groupby_month_data[data.columns+'_mean'] = temp_month.groupby(['month']).mean()
groupby_month_data[data.columns+'_max'] = temp_month.groupby(['month']).max()
groupby_month_data[data.columns+'_std'] = temp_month.groupby(['month']).std()
groupby_month_data

,channel_a_mean,channel_b_mean,channel_c_mean,channel_a_max,channel_b_max,channel_c_max,channel_a_std,channel_b_std,channel_c_std
month,,,,,,,,,
2,155.268366,345.783901,NaN,230.210007,457.399994,NaN,24.681381,55.683073,NaN
3,177.955931,354.392811,187.650578,319.000000,495.799988,303.329987,31.261458,57.311530,35.770876
4,178.605306,360.468998,183.858699,322.309998,530.150024,288.220001,35.187174,62.983482,33.047358
5,183.852507,368.224958,191.776504,258.019989,526.599976,260.739990,27.447445,62.082308,30.980661
6,185.313404,384.381230,192.968352,322.959991,538.369995,241.550003,32.117296,70.403536,29.093475
7,191.553842,372.643712,182.817390,304.959991,575.570007,238.500000,30.589956,60.564336,23.322371
8,175.774832,346.026875,186.220308,237.240005,470.529999,246.979996,23.860643,55.563326,24.261246
9,172.784686,346.728912,195.680492,230.619995,494.390015,247.000000,27.844160,64.105435,24.977783
10,181.469373,348.569423,201.485997,233.669998,489.500000,250.149994,28.586387,62.779205,24.719637


Group by hour with mean, max and std

In [22]:
temp_hour = data.copy()
temp_hour['hour'] = data.index.hour

groupby_hour_data = pd.DataFrame()
groupby_hour_data[data.columns+'_mean'] = temp_hour.groupby(['hour'], ).mean()
groupby_hour_data[data.columns+'_max'] = temp_hour.groupby(['hour']).max()
groupby_hour_data[data.columns+'_std'] = temp_hour.groupby(['hour']).std()
groupby_hour_data

,channel_a_mean,channel_b_mean,channel_c_mean,channel_a_max,channel_b_max,channel_c_max,channel_a_std,channel_b_std,channel_c_std
hour,,,,,,,,,
0,166.238602,359.966178,184.540444,247.130005,499.660004,242.990005,17.042893,30.967091,15.794199
1,151.610506,324.140851,167.589180,233.089996,457.760010,212.350006,17.550698,29.265080,15.502972
2,141.311391,293.797605,154.794091,246.119995,427.320007,204.149994,18.471530,28.668752,15.397638
3,135.797488,271.547336,148.549785,291.750000,388.279999,190.119995,19.971304,28.720634,16.013739
4,134.937784,258.991561,146.265326,304.700012,385.049988,185.630005,20.734486,29.290639,16.099666
5,141.903999,256.438909,148.318541,305.029999,418.049988,191.449997,20.869483,30.300978,15.735371
6,160.953971,273.103870,158.131807,288.179993,439.239990,209.990005,22.999830,33.285555,17.181774
7,183.511881,306.137030,177.142828,291.070007,454.970001,240.210007,23.824700,38.828314,21.416482
8,192.455175,346.860561,196.858392,306.899994,493.679993,259.670013,18.566110,45.094581,20.860634


#### 6. Make plots for the mean values for grouped data by the hour, month, year (from previous point).

There is only one year in the data so there is no point in making a graph. 

Plot for group by month

In [23]:
channels_list = ['channel_a', 'channel_b', 'channel_c']
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05, subplot_titles=channels_list)
for i, channel_name in enumerate(channels_list):
    fig.append_trace(go.Scatter(x=groupby_month_data.index, 
                                y=groupby_month_data[channel_name+'_mean'], 
                                mode='lines', 
                                name=channel_name+'_mean',
                                marker=dict(color='red')), 
                                row=1+i, col=1)
    fig.append_trace(go.Scatter(x=groupby_month_data.index, 
                                y=groupby_month_data[channel_name+'_max'], 
                                mode='lines', 
                                name=channel_name+'_max',
                                marker=dict(color='blue')), 
                                row=1+i, col=1)
    fig.append_trace(go.Scatter(x=groupby_month_data.index, 
                                y=groupby_month_data[channel_name+'_std'], 
                                mode='lines', 
                                name=channel_name+'_std',
                                marker=dict(color='green')), 
                                row=1+i, col=1)
fig.update_layout(
    title={'text':"Statistics for data group by month", 
           'y':0.98, 'x':0.45, 
           'xanchor': 'center', 
           'font':dict(size=20)}
    )
fig.update_xaxes(title_text='month', row=1+i, col=1)
fig.write_html(path_to_save_plot + 'groupby_month.html')

Plot for group by hour

In [24]:
channels_list = ['channel_a', 'channel_b', 'channel_c']
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05, subplot_titles=channels_list)
for i, channel_name in enumerate(channels_list):
    fig.append_trace(go.Scatter(x=groupby_hour_data.index, 
                                y=groupby_hour_data[channel_name+'_mean'], 
                                mode='lines', 
                                name=channel_name+'_mean',
                                marker=dict(color='red')), 
                                row=1+i, col=1)
    fig.append_trace(go.Scatter(x=groupby_hour_data.index, 
                                y=groupby_hour_data[channel_name+'_max'], 
                                mode='lines', 
                                name=channel_name+'_max',
                                marker=dict(color='blue')), 
                                row=1+i, col=1)
    fig.append_trace(go.Scatter(x=groupby_hour_data.index, 
                                y=groupby_hour_data[channel_name+'_std'], 
                                mode='lines', 
                                name=channel_name+'_std',
                                marker=dict(color='green')), 
                                row=1+i, col=1)
fig.update_layout(
    title={'text':"Statistics for data group by hour", 
           'y':0.98, 'x':0.45, 
           'xanchor': 'center', 
           'font':dict(size=20)}
    )
fig.update_xaxes(title_text='hour', row=1+i, col=1)
fig.write_html(path_to_save_plot + 'groupby_hour.html')

#### 7. Create additional column with {0,1} values. 1 if given value from “channel_a” is greater than 250, otherwise 0.


In [25]:
data['channel_a_bigger_than_250'] = 0
data.loc[data['channel_a']>250, 'channel_a_bigger_than_250'] = 1

data[data['channel_a_bigger_than_250'] == 1]

,channel_a,channel_b,channel_c,channel_a_bigger_than_250
time,,,,
2023-03-17 10:25:00,250.490005,421.899994,185.779999,1
2023-03-17 10:30:00,251.899994,419.100006,184.259995,1
2023-03-17 10:35:00,252.460007,423.850006,184.789993,1
2023-03-17 10:40:00,254.380005,425.959991,186.270004,1
2023-03-17 10:45:00,255.259995,426.390015,187.830002,1
...,...,...,...,...
2023-07-30 21:25:00,251.940002,416.399994,188.289993,1
2023-07-30 21:30:00,252.059998,420.679993,186.690002,1
2023-07-30 21:35:00,251.669998,417.089996,186.169998,1


#### 8. Make a plot for “channel_a” data with values greater than 250 marked with different colour. 

In [26]:
points_to_plot = data[data['channel_a_bigger_than_250'] == 1]
fig = go.Figure()
fig.add_traces(go.Scatter(x=data.index, y=data['channel_a'], mode='lines', name='channel_a'))
fig.add_traces(go.Scatter(x=points_to_plot.index, y=points_to_plot['channel_a'], mode='markers', name='channel_a > 250'))
fig.update_layout(
    title={'text':"Channel 'a' data", 'y':0.95, 'x':0.5, 'xanchor': 'center', 'font':dict(size=25)}, 
    xaxis_title={'text':'Time'}
    )
fig.write_html(path_to_save_plot + 'channel_a_bigger_than_250.html')

#### 9. Write a function that returns data from the date range specified in the function parameters (start_date, end_date). 

In [27]:
def get_data(data: pd.DataFrame, start_date: str, end_date: str) -> pd.DataFrame:
    temp = data.loc[start_date:end_date]
    return temp

In [28]:
get_data(data, '2023-03-17 10:25:00', '2023-03-17 10:40:00')

,channel_a,channel_b,channel_c,channel_a_bigger_than_250
time,,,,
2023-03-17 10:25:00,250.490005,421.899994,185.779999,1
2023-03-17 10:30:00,251.899994,419.100006,184.259995,1
2023-03-17 10:35:00,252.460007,423.850006,184.789993,1
2023-03-17 10:40:00,254.380005,425.959991,186.270004,1


#### 10. Write a function that returns the indexes of the points from the given column in Data Frame that are greater than mean + 2 * std.

In [29]:
def get_data_over_limit_indexes(data: pd.DataFrame, column: str) -> list:
    limit = data[column].mean() + 2*data[column].std()
    list_of_indexes = data[data[column] > limit].index.to_list()
    return list_of_indexes, limit

In [30]:
indexes, limit = get_data_over_limit_indexes(data, 'channel_a')
indexes[:10]

[Timestamp('2023-03-17 09:50:00'),
 Timestamp('2023-03-17 09:55:00'),
 Timestamp('2023-03-17 10:00:00'),
 Timestamp('2023-03-17 10:05:00'),
 Timestamp('2023-03-17 10:10:00'),
 Timestamp('2023-03-17 10:15:00'),
 Timestamp('2023-03-17 10:20:00'),
 Timestamp('2023-03-17 10:25:00'),
 Timestamp('2023-03-17 10:30:00'),
 Timestamp('2023-03-17 10:35:00')]

In [31]:
limit

240.28739103906844

#### 11. Make a plot with the marked points received from the function from the previous task. Add a line that marks the boundary for the marked points.

In [32]:
indexes, limit = get_data_over_limit_indexes(data, 'channel_a')
points_to_plot = data.loc[indexes]
fig = go.Figure()
fig.add_traces(go.Scatter(x=data.index, y=data['channel_a'], mode='lines', name='channel_a'))
fig.add_traces(go.Scatter(x=points_to_plot.index, y=points_to_plot['channel_a'], mode='markers', 
                          name='channel_a > limit'))
fig.add_traces(go.Scatter(x=data.index, y=[limit]*len(data), mode='lines', name='limit'))
fig.update_layout(
    title={'text':"Channel 'a' data", 'y':0.95, 'x':0.5, 'xanchor': 'center', 'font':dict(size=25)}, 
    xaxis_title={'text':'Time'}
    )
fig.write_html(path_to_save_plot + 'channel_a_bigger_than_limit.html')

#### 12. Save the data from channel_a to csv. The data in csv should have a frequency of 1h (it is necessary to make aggregation. e.g. mean). The data should also not have a value greater than 250 (replace with an empty value).

In [33]:
path_to_save_csv = './csv_files/'
Path(path_to_save_csv).mkdir(parents=True, exist_ok=True)

In [34]:
data_to_save = data['channel_a'].resample('1h').mean()
data_to_save[data_to_save > 250] = None
data_to_save.to_csv(path_to_save_csv + 'modified_channel_a.csv')